# Projeto Analytics V1 | PySpark | Canal do Youtube <a href="www.youtube.com/UnravelingCode">Unraveling Code</a>

### Importação de objetos

In [ ]:
import findspark
from pyspark.sql import SparkSession

### Configurações

In [ ]:
findspark.init()
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

# Analisando os dados

### Carregando os dados | source json

In [ ]:
path_json = '/Users/edsonlourenco/Downloads/source-4-ds-train.json'
df_imoveis = spark.read.json(path_json)

### Contando as linhas

In [ ]:
df_imoveis.count() #133.964***

### Checando o schema

In [ ]:
df_imoveis.printSchema()

### Selecionando as todas colunas | 10 linhas apenas

In [ ]:
df_imoveis.select('*').limit(10)

### Selecionando uma coluna

In [ ]:
df_imoveis.select('address.neighborhood').limit(10)

### Selecionando colunas e analisando amostragem de 10 linhas

In [ ]:
cols = [
         'address.country', 'address.city', 'address.neighborhood', 'address.state', 'address.street', 'address.zone', 
         'bathrooms', 'bedrooms', 'description', 'listingStatus', 'owner', 'parkingSpaces', 'pricingInfos.businessType',
         'pricingInfos.monthlyCondoFee','pricingInfos.period', 'pricingInfos.price', 'pricingInfos.rentalTotalPrice', 'pricingInfos.yearlyIptu',
         'publicationType', 'suites', 'title', 'totalAreas', 'unitTypes', 'usableAreas'
       ]

df_imoveis.select(cols).limit(10) 

### Buscando dados distintos | método distinct()

In [ ]:
df_imoveis.select('address.neighborhood').distinct()

### Contando dados agrupados e contando | método groupby() e count()

In [ ]:
df_imoveis.groupBy('address.neighborhood').count()

In [ ]:
df_imoveis.groupBy('unitTypes').count()

### Contando dados agrupados | método groupby() informando coluna

In [ ]:
from pyspark.sql import functions as F

df_imoveis \
        .groupBy('address.neighborhood') \
        .agg(F.count("*").alias("qty"))

### Melhorando identação do código

In [ ]:
from pyspark.sql import functions as F #repetido apenas para fins didáticos

(
    df_imoveis
            .groupBy(F.col("address.neighborhood"))
            .agg(F.count("*").alias("qty"))
)

### Pesquisando por uma coluna | método where()

In [ ]:
from pyspark.sql import functions as F #repetido apenas para fins didáticos

(
    df_imoveis
            .where(F.col("address.neighborhood") == "Jardim Coimbra")
            .select("*")
)

### Pesquisando por uma coluna | método where() | por contagem

In [ ]:
from pyspark.sql import functions as F #repetido apenas para fins didáticos

df_imoveis_contagem = (
                        df_imoveis
                                .groupBy(F.col("address.neighborhood"))
                                .agg(F.count("*").alias("qty"))
)

(
    df_imoveis_contagem
            .where(F.col("qty") > 10)
            .select("*")
)

### Contando dados agrupados com mais de uma coluna | método groupBy()

In [ ]:
from pyspark.sql import functions as F #repetido apenas para fins didáticos

(
    df_imoveis
            .groupBy(
                        F.col("address.neighborhood"),
                        F.col("unitTypes")
                     )
            .agg(F.count("*").alias("qty"))
)

### Contando dados agrupados com mais de uma coluna | método groupBy() com orderBy()

In [ ]:
from pyspark.sql import functions as F #repetido apenas para fins didáticos

(
    df_imoveis
            .groupBy(
                        F.col("address.neighborhood"),
                        F.col("unitTypes")
                     )
            .agg(F.count("*").alias("qty"))
            .orderBy(F.col("address.neighborhood"))
)

### Consultando por casas e trazendo dados financieros | criar dataframe de casas para locação

In [ ]:
from pyspark.sql import functions as F #repetido apenas para fins didáticos

tipos_imoveis = ['FARM', 'HOME', 'COUNTRY_HOUSE']

cols = [
         'address.country', 'address.city', 'address.neighborhood', 'address.state', 'address.street', 'address.zone', 
         'bathrooms', 'bedrooms', 'description', 'listingStatus', 'owner', 'parkingSpaces', 'pricingInfos.businessType',
         'pricingInfos.monthlyCondoFee','pricingInfos.period', 'pricingInfos.price', 'pricingInfos.rentalTotalPrice', 'pricingInfos.yearlyIptu',
         'publicationType', 'suites', 'title', 'totalAreas', 'unitTypes', 'usableAreas'
       ]

(
      df_imoveis.select(cols)
                           .where(
                              (F.col("unitTypes").isin(tipos_imoveis)) &
                              (F.col("pricingInfos.businessType") == "RENTAL")
                           )
)